In [1]:
import pandas as pd 
df = pd.read_csv("/kaggle/input/qadataset/100_Unique_QA_Dataset.csv")

In [2]:
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [4]:
# tokenize

def tokenize(text):
    text = text.lower()
    text = text.replace('?','')
    text = text.replace("'","")
    return text.split()

In [5]:
tokenize("What's your name?")

['whats', 'your', 'name']

In [6]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [7]:
# vocab 
vocab = {'<UNK>':0}


In [8]:
def build_vocab(row):
    tokenized_question = tokenize(row['question'])
    tokenized_answer = tokenize(row['answer'])

    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token]=len(vocab)

In [9]:
df.apply(build_vocab,axis=1)

0     None
1     None
2     None
3     None
4     None
      ... 
85    None
86    None
87    None
88    None
89    None
Length: 90, dtype: object

In [10]:
len(vocab)

324

In [11]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [12]:
# convert words to numerical indices

def text_to_indices(text,vocab):

    indexed_text = []

    for token in tokenize(text):

        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab['<UNK>'])

    return indexed_text

In [13]:
text_to_indices("What is Rp study point",vocab)

[1, 2, 0, 221, 25]

In [14]:
import torch 
from torch.utils.data import Dataset, DataLoader

In [15]:
class CustomDataset(Dataset):

    def __init__(self,df,vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self,index):

        numerical_question = text_to_indices(self.df.iloc[index]['question'],self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)
        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [16]:
dataset = CustomDataset(df,vocab)

In [17]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [19]:
for question,answer in dataloader:
    print(question,answer[0])

tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([121])
tensor([[ 10,  75, 111]]) tensor([112])
tensor([[ 42, 263, 264,  14, 265, 266, 158, 267]]) tensor([268])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([9])
tensor([[  1,   2,   3,   4,   5, 236, 237]]) tensor([238])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([205])
tensor([[ 10,  96,   3, 104, 239]]) tensor([240])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([124])
tensor([[ 78,  79, 261, 151,  14, 262, 153]]) tensor([36])
tensor([[  1,   2,   3,   4,   5, 286]]) tensor([287])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([36])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([166])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([58])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([298])
tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([149])
tensor([[42, 43, 44, 45

In [20]:
import torch.nn as nn

In [21]:
class SimpleRNN(nn.Module):

    def __init__(self,vocab_size):

        super().__init__()
        self.embedding = nn.Embedding(vocab_size,embedding_dim=50)
        self.rnn = nn.RNN(50,64,batch_first=True)
        self.fc = nn.Linear(64,vocab_size)

    def forward(self,question):
        embedded_question = self.embedding(question)
        hidden, final = self.rnn(embedded_question)
        output = self.fc(final.squeeze(0))
        return output

In [26]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [27]:
learning_rate = 0.001
epochs = 20

In [28]:
model = SimpleRNN(len(vocab))

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [30]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 523.420075
Epoch: 2, Loss: 456.454120
Epoch: 3, Loss: 377.953644
Epoch: 4, Loss: 317.057743
Epoch: 5, Loss: 265.100234
Epoch: 6, Loss: 217.611086
Epoch: 7, Loss: 174.244534
Epoch: 8, Loss: 135.625684
Epoch: 9, Loss: 104.002087
Epoch: 10, Loss: 79.147776
Epoch: 11, Loss: 60.887804
Epoch: 12, Loss: 47.433460
Epoch: 13, Loss: 37.417863
Epoch: 14, Loss: 30.444482
Epoch: 15, Loss: 25.072430
Epoch: 16, Loss: 20.698071
Epoch: 17, Loss: 17.448427
Epoch: 18, Loss: 14.870777
Epoch: 19, Loss: 12.717108
Epoch: 20, Loss: 11.019878


In [32]:
for epoch in range(epochs):
    total_loss = 0
    num_batches = 0

    for question, answer in dataloader:
        optimizer.zero_grad()
        output = model(question)
        loss = criterion(output, answer[0])
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    print(f"Epoch: {epoch + 1}, Avg Loss: {avg_loss:.4f}")


Epoch: 1, Avg Loss: 0.1074
Epoch: 2, Avg Loss: 0.0946
Epoch: 3, Avg Loss: 0.0838
Epoch: 4, Avg Loss: 0.0746
Epoch: 5, Avg Loss: 0.0669
Epoch: 6, Avg Loss: 0.0603
Epoch: 7, Avg Loss: 0.0546
Epoch: 8, Avg Loss: 0.0498
Epoch: 9, Avg Loss: 0.0456
Epoch: 10, Avg Loss: 0.0417
Epoch: 11, Avg Loss: 0.0383
Epoch: 12, Avg Loss: 0.0352
Epoch: 13, Avg Loss: 0.0326
Epoch: 14, Avg Loss: 0.0302
Epoch: 15, Avg Loss: 0.0279
Epoch: 16, Avg Loss: 0.0260
Epoch: 17, Avg Loss: 0.0242
Epoch: 18, Avg Loss: 0.0226
Epoch: 19, Avg Loss: 0.0211
Epoch: 20, Avg Loss: 0.0197


In [41]:
def predict(model,question,threshold=0.5):

    # convert question to numbers
    numerical_question = text_to_indices(question, vocab)
   
    # tensor 
    question_tensor = torch.tensor(numerical_question).unsqueeze(0)
    
    # send to model
    output = model(question_tensor)

    # convert logits into probs
    probs = torch.nn.functional.softmax(output,dim=1)

    # find index of max prob
    value, index = torch.max(probs, dim=1)

    if value< threshold:
        print("I don't know")
    else:
        print(list(vocab.keys())[index])

In [42]:
predict(model, "What is the largest planet in our solar system?")

jupiter


In [43]:
list(vocab.keys())[7]

'paris'

In [44]:
list(vocab.keys())

['<UNK>',
 'what',
 'is',
 'the',
 'capital',
 'of',
 'france',
 'paris',
 'germany',
 'berlin',
 'who',
 'wrote',
 'to',
 'kill',
 'a',
 'mockingbird',
 'harper-lee',
 'largest',
 'planet',
 'in',
 'our',
 'solar',
 'system',
 'jupiter',
 'boiling',
 'point',
 'water',
 'celsius',
 '100',
 'painted',
 'mona',
 'lisa',
 'leonardo-da-vinci',
 'square',
 'root',
 '64',
 '8',
 'chemical',
 'symbol',
 'for',
 'gold',
 'au',
 'which',
 'year',
 'did',
 'world',
 'war',
 'ii',
 'end',
 '1945',
 'longest',
 'river',
 'nile',
 'japan',
 'tokyo',
 'developed',
 'theory',
 'relativity',
 'albert-einstein',
 'freezing',
 'fahrenheit',
 '32',
 'known',
 'as',
 'red',
 'mars',
 'author',
 '1984',
 'george-orwell',
 'currency',
 'united',
 'kingdom',
 'pound',
 'india',
 'delhi',
 'discovered',
 'gravity',
 'newton',
 'how',
 'many',
 'continents',
 'are',
 'there',
 'on',
 'earth',
 '7',
 'gas',
 'do',
 'plants',
 'use',
 'photosynthesis',
 'co2',
 'smallest',
 'prime',
 'number',
 '2',
 'invented'